<a href="https://colab.research.google.com/github/ganeshmp01/DS-Classwork/blob/main/Transformer_1_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.2 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata

dataset = pd.read_csv("/content/movie_reviews.csv.bz2")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [5]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
dataset['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [7]:
def strip_html_tags(text):
  soup = BeautifulSoup(text,'html.parser')
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+','\n',stripped_text)
  return stripped_text

In [8]:
strip_html_tags(dataset['review'][20])

"After the success of Die Hard and it's sequels it's no surprise really that in the 1990s, a glut of 'Die Hard on a .....' movies cashed in on the wrong guy, wrong place, wrong time concept. That is what they did with Cliffhanger, Die Hard on a mountain just in time to rescue Sly 'Stop or My Mom Will Shoot' Stallone's career.Cliffhanger is one big nit-pickers dream, especially to those who are expert at mountain climbing, base-jumping, aviation, facial expressions, acting skills. All in all it's full of excuses to dismiss the film as one overblown pile of junk. Stallone even managed to get out-acted by a horse! However, if you an forget all the nonsense, it's actually a very lovable and undeniably entertaining romp that delivers as plenty of thrills, and unintentionally, plenty of laughs.You've got to love John Lithgows sneery evilness, his tick every box band of baddies, and best of all, the permanently harassed and hapless 'turncoat' agent, Rex Linn as Travers.He may of been Henry in

In [9]:
def remove_accented_characters(text):
  text = unicodedata.normalize("NFKD",text).encode("ascii",'ignore').decode('utf-8','ignore')
  return text

In [10]:
def pre_process_corpus(texts):
  preprocessed_text = []
  for doc in tqdm.tqdm(texts):
    doc = strip_html_tags(doc)
    #doc = doc.translate(doc.maketrans("\n\t\r",'   '))
    doc = doc.lower()
    doc = remove_accented_characters(doc)
    #Do remove contractions as well and check the results
    doc = re.sub(r'[^a-zA-Z0-9\s]','',doc,re.I|re.A)
    doc = re.sub(r' +',' ',doc)
    doc = doc.strip()
    #doc = "".join(doc)
    preprocessed_text.append(doc)
  return preprocessed_text

In [11]:
reviews  = dataset['review'].values
sentiment = dataset['sentiment'].values

train_reviews = reviews[:35000]
train_sentiment = sentiment[:35000]

test_reviews = reviews[35000:]
test_sentiment = sentiment[35000:]

In [12]:
pre_process_corpus([dataset['review'][20]])

100%|██████████| 1/1 [00:00<00:00, 752.75it/s]


['after the success of die hard and its sequels its no surprise really that in the 1990s a glut of die hard on a movies cashed in on the wrong guy wrong place wrong time concept that is what they did with cliffhanger die hard on a mountain just in time to rescue sly stop or my mom will shoot stallones careercliffhanger is one big nitpickers dream especially to those who are expert at mountain climbing basejumping aviation facial expressions acting skills all in all its full of excuses to dismiss the film as one overblown pile of junk stallone even managed to get outacted by a horse however if you an forget all the nonsense its actually a very lovable and undeniably entertaining romp that delivers as plenty of thrills and unintentionally plenty of laughsyouve got to love john lithgows sneery evilness his tick every box band of baddies and best of all the permanently harassed and hapless turncoat agent rex linn as travershe may of been henry in portrait of a serial killer but michael roo

In [13]:
%%time
norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

  2%|▏         | 725/35000 [00:00<00:09, 3642.80it/s]<ipython-input-7-46650b581e77>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text,'html.parser')
100%|██████████| 15000/15000 [00:04<00:00, 3516.75it/s]

CPU times: user 14 s, sys: 123 ms, total: 14.1 s
Wall time: 14.5 s


In [14]:
import tensorflow as tf

t = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',oov_token = 'UNK')

In [15]:
t.fit_on_texts(norm_train_reviews)

In [16]:
t.word_index['UNK'] = 0

In [17]:
train_sequences = t.texts_to_sequences(norm_train_reviews)
test_sequences = t.texts_to_sequences(norm_test_reviews)

In [18]:
train_sequences[0]

[28,
 5,
 2,
 78,
 1898,
 43,
 1048,
 12,
 101,
 145,
 40,
 521,
 3151,
 409,
 441,
 26,
 3210,
 34,
 23,
 204,
 14,
 10,
 7,
 576,
 47,
 592,
 15,
 7117,
 87,
 146,
 12,
 3221,
 69,
 42,
 3151,
 13,
 29,
 5505,
 3,
 15732,
 132,
 5,
 577,
 60,
 286,
 8,
 204,
 36,
 2,
 665,
 138,
 1730,
 69,
 10,
 7,
 21,
 4,
 118,
 16,
 2,
 8205,
 5259,
 39,
 11876,
 10,
 118,
 2383,
 55,
 5646,
 15,
 5347,
 6,
 1419,
 384,
 39,
 577,
 29,
 7,
 3375,
 8,
 2,
 356,
 337,
 5,
 2,
 68956,
 7,
 466,
 3151,
 14,
 12,
 7,
 2,
 11388,
 340,
 6,
 2,
 14030,
 6160,
 2528,
 1066,
 51153,
 9,
 2612,
 1386,
 20,
 22197,
 525,
 33,
 4855,
 2568,
 5,
 2,
 1159,
 115,
 31,
 2,
 7044,
 25,
 2869,
 11877,
 3,
 405,
 51154,
 38,
 17279,
 7,
 21,
 320,
 20,
 2,
 5060,
 3482,
 525,
 7,
 345,
 6,
 68957,
 7645,
 37120,
 13025,
 4606,
 7904,
 2448,
 3,
 15285,
 51155,
 325,
 9221,
 6727,
 11878,
 3,
 8666,
 33347,
 23,
 108,
 234,
 19393,
 56,
 130,
 2,
 269,
 1255,
 5,
 2,
 118,
 7,
 663,
 6,
 2,
 186,
 12,
 9,
 262,
 11

In [19]:
len(t.word_index)

176890

In [20]:
t.document_count

35000

In [21]:
MAX_SEQUENCE_LENGTH = 1000

In [22]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(train_sequences,maxlen = MAX_SEQUENCE_LENGTH)
X_test = tf.keras.preprocessing.sequence.pad_sequences(test_sequences,maxlen = MAX_SEQUENCE_LENGTH)

In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
num_classes = 2

In [24]:
y_train = le.fit_transform(train_sentiment)
y_test = le.transform(test_sentiment)

In [25]:
VOCAB_SIZE = len(t.word_index)

In [26]:
EMBEDDING_DIM = 300

In [27]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim = VOCAB_SIZE,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCE_LENGTH))
model.add(tf.keras.layers.GRU(128,return_sequences=False))
model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [28]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [29]:
model.fit(X_train,y_train,epochs =2,batch_size = 100,shuffle = True,validation_data=(X_test,y_test))

Epoch 1/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 45s 107ms/step - accuracy: 0.6971 - loss: 0.5501 - val_accuracy: 0.7817 - val_loss: 0.5387
Epoch 2/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 38s 108ms/step - accuracy: 0.9331 - loss: 0.1782 - val_accuracy: 0.8291 - val_loss: 0.4702


In [30]:
score = model.evaluate(X_test,y_test)
print("Accuracy: ",score[1]*100)

469/469 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.8308 - loss: 0.4675
Accuracy:  82.90666937828064


In [32]:
embedding_layer = model.get_layer('embedding')

In [33]:
embedding_weights = embedding_layer.get_weights()[0]

word_index = t.word_index
word_embeddings = {}

for word, idx in word_index.items():
  if idx <VOCAB_SIZE:
    word_embeddings[word] = embedding_weights[idx]

for word, embedding in list(word_embeddings.items())[10:15]:
  print(f'Word: {word}, Embedding: {embedding}')

Word: i, Embedding: [-8.17499775e-03 -3.83341014e-02  1.28292963e-02  1.13458615e-02
 -1.94527041e-02 -5.92644028e-02  1.14658838e-02 -9.80358105e-03
  4.80897762e-02 -9.57615615e-04  5.22432998e-02  7.58719305e-03
 -1.40462909e-02  9.82443895e-03 -3.89755219e-02 -4.60628681e-02
  3.36208045e-02 -2.47021276e-03 -1.40996854e-04  3.60901505e-02
  3.87570448e-03  4.55788374e-02 -9.39479098e-03  1.39320293e-03
 -5.83516732e-02 -6.28049001e-02  2.62052752e-02  1.72788948e-02
  3.10680451e-04  4.03640829e-02  8.65538605e-03 -7.08986968e-02
  2.41142418e-02  4.08895984e-02  5.87493703e-02 -3.20889913e-02
 -3.74916643e-02 -2.28655469e-02 -3.05113867e-02  4.79947543e-04
  1.66650116e-02 -4.96121421e-02  3.51254977e-02  3.61123867e-02
  2.46633135e-04  1.66024994e-02  3.58334519e-02  2.76589836e-03
 -2.17476636e-02 -8.23515188e-03  8.80961679e-03 -2.94150189e-02
 -2.93761380e-02  3.92874442e-02 -7.23479316e-02  3.54735255e-02
  3.52111682e-02 -5.62608801e-02  7.85708614e-03 -2.16294173e-02
  1.1

In [34]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim = VOCAB_SIZE,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCE_LENGTH))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=False)))
model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])

In [35]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [36]:
model.fit(X_train,y_train,epochs =2,batch_size = 100,shuffle = True,validation_data=(X_test,y_test))

Epoch 1/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 82s 221ms/step - accuracy: 0.7364 - loss: 0.5003 - val_accuracy: 0.8751 - val_loss: 0.3080
Epoch 2/2
350/350 ━━━━━━━━━━━━━━━━━━━━ 79s 213ms/step - accuracy: 0.9383 - loss: 0.1701 - val_accuracy: 0.8621 - val_loss: 0.3203


In [37]:
score = model.evaluate(X_test,y_test)
print("Accuracy: ",score[1]*100)

469/469 ━━━━━━━━━━━━━━━━━━━━ 15s 31ms/step - accuracy: 0.8609 - loss: 0.3216
Accuracy:  86.20666861534119


In [38]:
embedding_layer = model.get_layer('embedding_1')

In [40]:
embedding_weights = embedding_layer.get_weights()[0]

word_index = t.word_index
word_embeddings = {}

for word, idx in word_index.items():
  if idx <VOCAB_SIZE:
    word_embeddings[word] = embedding_weights[idx]

for word, embedding in list(word_embeddings.items())[10:15]:
  print(f'Word: {word}, Embedding: {embedding}')

Word: i, Embedding: [-0.01933309  0.00983655  0.02937908  0.02770332  0.00853987 -0.04810513
 -0.04153784  0.00437329 -0.00268954  0.04479955  0.01713385  0.02009713
 -0.0248018   0.03004493 -0.02760215 -0.0008151  -0.01668525  0.00704711
 -0.0391495  -0.04282119 -0.01791816 -0.01869473  0.00279625  0.0265147
  0.02833105  0.04500419  0.00160232  0.03266828  0.0412422   0.00333807
 -0.0358512   0.00144445  0.03990177 -0.01579686  0.04552811  0.03480966
  0.01834105 -0.02737903  0.00567645 -0.0433968  -0.04538808  0.00216982
  0.02498676 -0.02062954  0.07413612 -0.0039865  -0.00451474 -0.02949022
 -0.03040405  0.00152148  0.04564925  0.02362524  0.02948887 -0.03584387
  0.0169432   0.00996579 -0.00559121  0.06845444  0.02827549  0.0097586
 -0.02160517  0.02903984 -0.03494948 -0.01990704  0.025696    0.02937804
 -0.01893311  0.00951342  0.01926052 -0.03270765  0.04001921  0.02111686
 -0.03277399 -0.0410438   0.03076111 -0.00029196 -0.03458452  0.04143754
 -0.02187392 -0.01870119 -0.03874